In [2]:
import pandas as pd

In [23]:
def hum_dis(seq1, seq2):
    seq1 = seq1.upper()
    seq2 = seq2.upper()
    if len(seq1) != len(seq2):
        # print("seq_not_match")
        # print(seq1, seq2)
        return -1
    else:
        cont = 0
        for char in range(len(seq1)):
            if seq1[char] != seq2[char]:
                cont += 1
        return cont
    

def create_seq_lib(
    seq_list,
    color_fraction={
        "Red": [_ / 4 for _ in range(5)],
        "Green": [_ / 2 for _ in range(3)],
        "Blue": [_ / 4 for _ in range(5)],
        "Yellow": [_ / 4 for _ in range(5)],
    },
):
    seq_lib = pd.DataFrame(columns=["seq", "color", "grade", "fraction"])
    color_list = []
    for color in color_fraction.keys():
        color_list += [color] * len(color_fraction[color])
    seq_lib["color"] = color_list

    if len(seq_lib["color"]) > len(seq_list):
        print("Seq Not Enough")
        return ValueError

    seq_lib["seq"] = seq_list[: len(seq_lib["color"])]

    fra = []
    grade = []
    for color in color_fraction.keys():
        fra += color_fraction[color]
        grade += [_ for _ in range(len(color_fraction[color]))]
    seq_lib["fraction"] = fra
    seq_lib["grade"] = grade

    return seq_lib


import itertools

def create_barcode_lib(
    seq_lib,
    color_order=["Green", "Red", "Blue", "Yellow"],
    sum_num=5,
    sum_list=["Red", "Yellow", "Blue"],
):
    color_order_seq = [_ + "seq" for _ in color_order]
    barcode_lib = pd.DataFrame(columns=["barcode"] + color_order + color_order_seq)

    grade_list = {
        color: list(seq_lib[seq_lib.color == color].grade.unique())
        for color in color_order
    }
    barcode_lib[color_order] = list(itertools.product(*grade_list.values()))

    for i in range(len(barcode_lib)):
        grade = barcode_lib.loc[i, color_order]

        barcode_sub_list = [
            list(
                seq_lib[
                    (seq_lib.color == color_order[_]) & (seq_lib.grade == grade[_])
                ].seq
            )[0]
            for _ in range(len(color_order))
        ]
        barcode_lib.loc[i, color_order_seq] = barcode_sub_list
        barcode_lib.loc[i, "barcode"] = "".join(barcode_sub_list)

    if sum_num:
        barcode_lib["sum"] = barcode_lib[sum_list].sum(axis=1)
        barcode_lib = barcode_lib[
            # (barcode_lib["sum"] >= 1) 
            # &
            (barcode_lib["sum"] == sum_num)
        ]
        barcode_lib.set_index('barcode', inplace=True)

    return barcode_lib

In [ ]:
file = './random_seq_filtered.txt'
with open(file, "r") as f:
    seq_list = f.readlines()
seq_list = [seq_list[_].replace('\n','') for _ in range(1,len(seq_list),2)]

In [4]:
seq_lib = create_seq_lib(
    seq_list=seq_list,
    color_fraction={
        "Green": [0, 0.5, 1],
        "Red": [_ / 5 for _ in range(6)],
        "Blue": [_ / 5 for _ in range(6)],
        "Yellow": [_ / 5 for _ in range(6)],
    },
)

In [46]:
barcode_lib = create_barcode_lib(
    seq_lib,
    color_order=["Green", "Red", "Blue", "Yellow"],
    sum_num=5,
    sum_list=["Red", "Yellow", "Blue"],
)

In [ ]:
seq_lib.to_csv(r"./seq_lib.csv")
barcode_lib.to_csv(r"./barcode_lib.csv")